In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
dataset = GermanDataset(device=device)

In [4]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = MLP(num_features=dataset.XZ_train.shape[1],n_layers=[200,200,200,200])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=False, 
            delta_effort=hp['delta_effort'],
            effort_iter=hp['effort_iter'],
            effort_lr=hp['effort_lr']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds):
    
    SGD_train, _, SGD_test = lr_kde_model_runner(dataset, SGD_hp, seeds)
    EI_fc_train, _, EI_fc_test = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    EI_kde_train, _, EI_kde_test = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    EI_fb_train, _, EI_fb_test = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

#### SGD Hyperparameter Selection

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 50
SGD_hp_test['batch_size'] = 256
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 1.1
SGD_hp_test['effort_iter'] = 20
SGD_hp_test['effort_lr'] = 15

kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 15.93epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 16.84epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 19.66epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 22.95epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001          0          0.770312         0.8125   0.075      0.0615385  0.0674282  0.0674282
         0.001           0          0.984375         0.83125  0.145122   0.0514423  0.0854827  0.0877193
         0.01            0          0.975            0.8375   0.0735294  0.0336538  0.0541636  0.0689223
         0.1             0          0.307812         0.2625   0          0          0          0


The learning rate is decided as 0.01.


In [6]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.0001
SGD_hp['lambda_'] = 0


#### EI Hyperparameter Test

In [20]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.2, 0.3, 0.4, 0.5]
EI_hp_test['fairness'] = 'EI'

kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 15.50epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 16.50epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 17.86epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 16.65epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei          dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  ----------  ----------  ---------
         0.0001        0            0.770312         0.8125   0.075      0.0615385   0.0674282   0.0674282
         0.0001        0.2          0.767188         0.825    0.0428571  0.0586538   0.0626382   0.0626382
         0.0001        0.3          0.770312         0.81875  0.002849   0.0466346   0.0493736   0.0493736
         0.0001        0.4          0.760938         0.8125   0.05       0.0134615   0.00957996  0.0125313
         0.0001        0.5          0.760938         0.81875  0.047619   0.00721154  0.00957996  0.0363409


The lambda for EI is decided as 0.4.

In [21]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.0001
EI_hp_kde['lambda_'] = 0.3

In [9]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95, 0.99]
EI_hp_test['fairness'] = 'EI'

fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 15.62epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 14.79epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 17.56epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 19.89epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp          eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ----------  ---------
         0.0001       0             0.770312         0.8125   0.075      0.0615385  0.0674282   0.0674282
         0.0001       0.6           0.767188         0.8125   0.075      0.0615385  0.0674282   0.0674282
         0.0001       0.8           0.764062         0.81875  0.0709677  0.0524038  0.0541636   0.0541636
         0.0001       0.9           0.764062         0.83125  0.0418719  0.0495192  0.0493736   0.0493736
         0.0001       0.95          0.757812         0.825    0.0333333  0.0432692  0.040899    0.040899
         0.0001       0.99          0.75625          0.79375  0.0576923  0.0197115  0.00478998  0.10401


In [10]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.0001
EI_hp_fc['lambda_'] = 0.95

In [11]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.0001]
EI_hp_test['lambda_'] = [0, 0.1, 0.2, 0.3, 0.4]
EI_hp_test['fairness'] = 'EI'

fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 16.23epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:03<00:00, 14.49epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 17.59epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 50/50 [00:02<00:00, 16.99epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei          dp          eo       eodd
---------------  ---------  ----------------  --------------  ----------  ----------  ----------  ---------
         0.0001        0            0.770312         0.8125   0.075       0.0615385   0.0674282   0.0674282
         0.0001        0.1          0.767188         0.825    0.0666667   0.0432692   0.040899    0.040899
         0.0001        0.2          0.76875          0.81875  0.0826211   0.0466346   0.0276345   0.0501253
         0.0001        0.3          0.759375         0.8125   0.00909091  0.0163462   0.0228445   0.0363409
         0.0001        0.4          0.757812         0.81875  0.210084    0.00144231  0.00478998  0.0413534


In [12]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.0001
EI_hp_fb['lambda_'] = 0.3

#### Model training

In [22]:
seeds = np.arange(1,6)

SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds)

/var/folders/mt/mgpv1mld71dfr_kn_fq57str0000gn/T/ipykernel_78803/1840642636.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training: 100%|██████████| 50/50 [00:02<00:00, 20.08epochs/s]


training seed 2 started


Training: 100%|██████████| 50/50 [00:02<00:00, 21.68epochs/s]


...

training seed 4 started


Training: 100%|██████████| 50/50 [00:04<00:00, 11.97epochs/s]


training seed 5 started


Training: 100%|██████████| 50/50 [00:03<00:00, 14.70epochs/s]

Training finished for all seeds.


In [23]:
result = []
models = ["SGD train", "EI FC train", "EI KDE train", "EI FB train","SGD test", "EI FC test", "EI KDE test", "EI FB test"]
sol = [SGD_train, EI_fc_train, EI_kde_train, EI_fb_train, SGD_test, EI_fc_test, EI_kde_test, EI_fb_test]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model           accuracy_mean    accuracy_var    ei_mean      ei_var    dp_mean      dp_var    eo_mean      eo_var    eodd_mean    eodd_var
------------  ---------------  --------------  ---------  ----------  ---------  ----------  ---------  ----------  -----------  ----------
SGD train            0.7825        0.00435262  0.022358   0.0172625   0.0556652  0.0123467   0.0325433  0.00536577    0.0426083   0.0123416
EI FC train          0.7675        0.00318689  0.0180366  0.0114192   0.0570379  0.00772576  0.0371707  0.00744949    0.0464285   0.0103754
EI KDE train         0.775         0.00850092  0.0181615  0.00999633  0.054094   0.00741443  0.0323602  0.0018829     0.0411777   0.0124729
EI FB train          0.768125      0.0110309   0.0153517  0.013378    0.047514   0.00503869  0.0295466  0.00666247    0.0398629   0.0109014
SGD test             0.782         0.0102956   0.060305   0.0323776   0.0643333  0.0193352   0.0615176  0.0163751     0.0615176   0.0163751
EI FC test          